# 用 RSA 加密法傳遞字串並解密還原 

### import 加密法需要的套件

In [60]:
import numpy as np
import sympy as sym
import sympy.ntheory as ntheory 
import sympy.ntheory.residue_ntheory as residue_ntheory

### RSA加密
用RSA產生公鑰(e,N)及私鑰(d,N) 可參考 [reference](https://zh.wikipedia.org/wiki/RSA%E5%8A%A0%E5%AF%86%E6%BC%94%E7%AE%97%E6%B3%95)

In [61]:
def key_generate(n):
    a = int(n*np.random.rand(1))
    b = int(n*np.random.rand(1))
    p = sym.prime(a)
    q = sym.prime(b)
    N = p*q
    r = (p-1)*(q-1)
    gcd = 0
    while gcd!=1:
        e = int(r*np.random.rand(1))
        gcd = residue_ntheory.igcd(e,r)
    d = residue_ntheory.mod_inverse(e,r)
    return (e, d, N)

### 將輸入的文字轉成數字字串,並分割成適當大小的片段

In [62]:
def translate(string, M, U, block):
    message = ''
    unzip = ''

    for i in string:
        message += str(ord(i))
        unzip  += str(len(str(ord(i))))

    for i in range(0,int(len(message)/block+1)):
        a = ''
        j = 0
        while len(a)<block:
            if i*block+j==len(message):
                break
            a += message[i*block+j]
            j += 1      
        if a!= '':
            M.append(a)
    last_block = j
    
    for i in range(0,int(len(unzip)/block+1)):
        a = ''
        j = 0
        while len(a)<block:
            if i*block+j==len(unzip):
                break
            a += unzip[i*block+j]
            j += 1      
        if a!= '':
            U.append(a)
    return last_block   

### 用公鑰(e,N)加密以及用私鑰(d,N)解密

In [63]:
def encode(P, P_prime, e, N):
    for i in P:
        i = int(i)
        P_prime.append(pow(i,e,N))
def decode(A, A_prime, d, N):
    for i in A:
        i = int(i)
        A_prime.append(pow(i,d,N))


### 將解密後得到的數字字串轉換回文字字串
注意在translate時有把字串轉成數字 所以0開頭的字串會自動去0 back_translate時把0補回來

In [64]:
def back_translate(M, U, block, last_block):
    sm = ''
    su = ''
    string = ''
    index = 0
    for i in M[:-1]:
        if len(str(i))<block:
            sm += '0'*(block-len(str(i)))+str(i)
        else:
            sm += str(i)
    if len(str(M[-1]))<last_block:
        sm += '0'*(last_block-len(str(M[-1])))+str(M[-1])
    else:
        sm += str(M[-1])
    for i in U:
        su += str(i)  
    for i in range(0, len(su)):
        end = index+int(su[i])
        string += chr(int(sm[index:end]))
        index += int(su[i])
    return (string)
#        print(index)

### Main:產生公私鑰

In [65]:
n = 1e9
(e,d, N)=key_generate(n)


### Main: 加密及解密

In [68]:
text = '    臣亮言：先帝創業未半，而中道崩殂。今天下三分，益州疲弊，此誠危急存亡之秋也。然侍衛之臣，不懈於內；忠志之士，\
忘身於外者，蓋追先帝之殊遇，欲報之於陛下也。誠宜開張聖聽，以光先帝遺德，恢弘志士之氣；不宜妄自菲薄，引喻失義，以塞忠諫\
之路也。宮中府中，俱為一體，陟罰臧否，不宜異同。若有作姦犯科，及為忠善者，宜付有司，論其刑賞，以昭陛下平明之治，不宜篇私\
，使內外異法也。\n    侍中、侍郎郭攸之、費褘、董允等，此皆良實，志慮忠純，是以先帝簡拔以遺陛下。愚以為宮中之事，事無大小，悉以\
咨之，然後施行，必能裨補闕漏，有所廣益。將軍向寵，性行淑均，曉暢軍事，試用於昔日，先帝稱之曰「能」，是以眾議舉寵為督。愚以\
為營中之事，悉以咨之，必能使行陣和睦，優劣得所。親賢臣，遠小人，此先漢所以興隆也；親小人，遠賢臣，此後漢所以傾頹也。先帝在\
時，每與臣論此事，未嘗不歎息痛恨於桓、靈也。侍中、尚書、長史；參軍，此悉貞良死節之臣也，願陛下親之信之，則漢室之隆，可計日\
而待也。\n    臣本布衣，躬耕於南陽，苟全性命於亂世，不求聞達於諸侯。先帝不以臣卑鄙，猥自枉屈，三顧臣於草廬之中，諮臣以當世之事\
，由是感激，遂許先帝以驅馳。後值傾覆，受任於敗軍之際，奉命於危難之間，爾來二十有一年矣！先帝知臣謹慎，故臨崩寄臣以大事也。\
受命以來，夙夜憂勤，恐託付不效，以傷先帝之明。故五月渡瀘，深入不毛。今南方已定，兵甲已足，當獎率三軍，北定中原，庶竭駑鈍，\
攘除奸凶，興復漢室，還於舊都；此臣所以報先帝而忠陛下之職分也。至於斟酌損益，進盡忠言，則攸之、褘、允之任也。\n    願陛下託臣以\
討賊興復之效；不效，則治臣之罪，以告先帝之靈。若無興德之言，則戮允等，以彰其慢。陛下亦宜自課，以諮諏善道，察納雅言，深追先\
帝遺詔，臣不勝受恩感激。\n    今當遠離，臨表涕泣，不知所云。'

TEXT = ''
M = []
M_encode = []
M_decode = []
U = []
U_encode = []
U_decode = []
block = len(str(N))-1
last_block = translate(text, M, U, block)

encode(M, M_encode, e, N)
encode(U, U_encode, e, N)

decode(M_encode, M_decode , d, N)
decode(U_encode, U_decode , d, N)

TEXT = back_translate(M_decode, U_decode, block, last_block)
print(TEXT)

#print(M_decode)
#print(U_decode)

    臣亮言：先帝創業未半，而中道崩殂。今天下三分，益州疲弊，此誠危急存亡之秋也。然侍衛之臣，不懈於內；忠志之士，忘身於外者，蓋追先帝之殊遇，欲報之於陛下也。誠宜開張聖聽，以光先帝遺德，恢弘志士之氣；不宜妄自菲薄，引喻失義，以塞忠諫之路也。宮中府中，俱為一體，陟罰臧否，不宜異同。若有作姦犯科，及為忠善者，宜付有司，論其刑賞，以昭陛下平明之治，不宜篇私，使內外異法也。
    侍中、侍郎郭攸之、費褘、董允等，此皆良實，志慮忠純，是以先帝簡拔以遺陛下。愚以為宮中之事，事無大小，悉以咨之，然後施行，必能裨補闕漏，有所廣益。將軍向寵，性行淑均，曉暢軍事，試用於昔日，先帝稱之曰「能」，是以眾議舉寵為督。愚以為營中之事，悉以咨之，必能使行陣和睦，優劣得所。親賢臣，遠小人，此先漢所以興隆也；親小人，遠賢臣，此後漢所以傾頹也。先帝在時，每與臣論此事，未嘗不歎息痛恨於桓、靈也。侍中、尚書、長史；參軍，此悉貞良死節之臣也，願陛下親之信之，則漢室之隆，可計日而待也。
    臣本布衣，躬耕於南陽，苟全性命於亂世，不求聞達於諸侯。先帝不以臣卑鄙，猥自枉屈，三顧臣於草廬之中，諮臣以當世之事，由是感激，遂許先帝以驅馳。後值傾覆，受任於敗軍之際，奉命於危難之間，爾來二十有一年矣！先帝知臣謹慎，故臨崩寄臣以大事也。受命以來，夙夜憂勤，恐託付不效，以傷先帝之明。故五月渡瀘，深入不毛。今南方已定，兵甲已足，當獎率三軍，北定中原，庶竭駑鈍，攘除奸凶，興復漢室，還於舊都；此臣所以報先帝而忠陛下之職分也。至於斟酌損益，進盡忠言，則攸之、褘、允之任也。
    願陛下託臣以討賊興復之效；不效，則治臣之罪，以告先帝之靈。若無興德之言，則戮允等，以彰其慢。陛下亦宜自課，以諮諏善道，察納雅言，深追先帝遺詔，臣不勝受恩感激。
    今當遠離，臨表涕泣，不知所云。


In [69]:
text = 'Build me a son, O Lord, who will be strong enough to know when he is weak, and brave enough to face himself when he is afraid;\n\
one who will be proud and unbending in honest defeat, and humble and gentle in victory.\n\n\
Build me a son whose wishes will not take the place of deeds; a son who will know Thee…… and that to know himself is the foundation stone of knowledge.\n\n\
Lead him, I pray, not in the path of ease and comfort, but under the stress and spur of difficulties and challenge.\n\
Here, let him learn to stand up in the storm; here let him learn compassion for those that fail.\n\n\
Build me a son whose heart will be clear, whose goal will be high, a son who will master himself  before he seeks to master other men;\n\
one who will reach into the future, yet never forget the past.\n\n\
And after all these things are his, add, I pray, enough of a sense of humor, so that he may always be serious, yet never take himself too seriously.\n\n\
Give him humility, so that he may always remember the simplicity of true greatness, the open mind of true wisdom and the meekness of true strength.\n\n\
Then, I, his father will dare to whisper, "I have not lived in vain.\n'

TEXT = ''
M = []
M_encode = []
M_decode = []
U = []
U_encode = []
U_decode = []
block = len(str(N))-1
last_block = translate(text, M, U, block)

encode(M, M_encode, e, N)
encode(U, U_encode, e, N)

decode(M_encode, M_decode , d, N)
decode(U_encode, U_decode , d, N)

TEXT = back_translate(M_decode, U_decode, block, last_block)
print(TEXT)

#print(M_decode)
#print(U_decode)


Build me a son, O Lord, who will be strong enough to know when he is weak, and brave enough to face himself when he is afraid;
one who will be proud and unbending in honest defeat, and humble and gentle in victory.

Build me a son whose wishes will not take the place of deeds; a son who will know Thee…… and that to know himself is the foundation stone of knowledge.

Lead him, I pray, not in the path of ease and comfort, but under the stress and spur of difficulties and challenge.
Here, let him learn to stand up in the storm; here let him learn compassion for those that fail.

Build me a son whose heart will be clear, whose goal will be high, a son who will master himself  before he seeks to master other men;
one who will reach into the future, yet never forget the past.

And after all these things are his, add, I pray, enough of a sense of humor, so that he may always be serious, yet never take himself too seriously.

Give him humility, so that he may always remember the simplicity of 